<a href="https://colab.research.google.com/github/AlbertMdz/Optimizacion-Evolutiva/blob/master/QAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
from math import *

dist_matrix = np.array(
    [[0, 1, 2, 3, 1, 2],
     [1, 0, 1, 2, 2, 1],
     [2, 1, 0, 1, 3, 2],
     [3, 2, 1, 0, 4, 3],
     [1, 2, 3, 4, 0, 1],
     [2, 1, 2, 3, 1, 0]])
     

flow_matrix = np.array(
    [[0, 5, 2, 4, 1, 0],
     [5, 0, 3, 0, 2, 2],
     [2, 3, 0, 0, 0, 0],
     [4, 0, 0, 0, 5, 2],
     [1, 2, 0, 5, 0, 10],
     [0, 2, 0, 2, 10, 0]])
     

size_population = 50
chromosome_len = 6            
g_max = 10  
Pm = 0.02
Pc = 0.5
np.random.seed(12345678)

def cros_over(parent_1, parent_2, cros_idx1, cros_idx2):
    if cros_idx1 > cros_idx2:
        cros_idx1,cros_idx2 = cros_idx2,cros_idx1

    n = chromosome_len

    index = list(range(cros_idx2+1,n)) + list(range(0, cros_idx1))
    child_1, child_2 = parent_1.copy(), parent_2.copy()
  
    child_1["chromosm"][index] = [p for p in parent_2["chromosm"] if p not in parent_1["chromosm"][cros_idx1:cros_idx2+1]]
    child_2["chromosm"][index] = [p for p in parent_1["chromosm"] if p not in parent_2["chromosm"][cros_idx1:cros_idx2+1]]

    child_1["cost"], child_2["cost"] = cost(child_1["chromosm"]), cost(child_2["chromosm"])

    return child_1, child_2

def mutatation_swap(individual, idx_1, idx_2):
    if idx_1 > idx_2:
        idx_1,idx_2 = idx_2,idx_1

    individual["cost"] = mutation_cost(individual["chromosm"], individual["cost"], idx_1, idx_2)
    individual["chromosm"][idx_1], individual["chromosm"][idx_2] = individual["chromosm"][idx_2], individual["chromosm"][idx_1]

def ga_run_main():

    length = chromosome_len
    size_pop = size_population

    solutions = np.zeros(g_max, dtype = np.int64)

    num_crosses = ceil(size_pop/2.0 * Pc)
    num_mutations = ceil(size_pop * length * Pm)
    
    len_chrom_str = str(length) + 'int'
    
    datType = np.dtype([('chromosm', len_chrom_str), ('cost', np.int64)])
    
    
    parents = np.zeros(size_pop, dtype = datType)   
    
    for indiv_soln in parents:
        indiv_soln["chromosm"] = np.random.permutation(length)
        indiv_soln["cost"] = cost(indiv_soln["chromosm"])

    parents.sort(order = "cost", kind = 'mergesort')
    print (parents)
    for g in range(g_max):
        
       # Tournament 
        contestant_Idx = np.empty(size_pop, dtype = np.int32)  
        
        for index in range(size_pop):
            contestant_Idx[index] = np.random.randint(low = 0, high = np.random.randint(1, size_pop))
            
        contestant_pairs = zip(contestant_Idx[0:2*num_crosses:2], contestant_Idx[1:2*num_crosses:2]) 
        print (contestant_pairs)
        # Crossover 
        children = np.zeros(size_pop, dtype = datType)  
        cross_points = np.random.randint(length, size = 2*num_crosses)
        
        for pairs, index1, index2 in zip(contestant_pairs, range(0,2*num_crosses,2), range(1,2*num_crosses,2)):
            children[index1], children[index2] = cros_over(parents[pairs[0]], parents[pairs[1]], cross_points[index1], cross_points[index2])
            
        children[2*num_crosses:] = parents[contestant_Idx[2*num_crosses:]].copy()

        
        mutant_children_Idx = np.random.randint(size_pop, size = num_mutations)
        mutant_allele = np.random.randint(length, size = 2*num_mutations)

        for index, gen_Index in zip(mutant_children_Idx, range(num_mutations)):
            mutatation_swap(children[index], mutant_allele[2*gen_Index], mutant_allele[2*gen_Index+1])

        
        children.sort(order = "cost", kind = 'mergesort')
        
        
        if children["cost"][0] > parents["cost"][0]:
           children[-1] = parents[0]
        
        parents = children  

        parents.sort(order = "cost", kind = 'mergesort')
        
        print("Costo de la mejor generación es", g, ":", int(parents[0]["cost"]))
        solutions[g] = parents[0]["cost"]

    return parents[0]["chromosm"], parents[0]["cost"]

def mutation_cost(current_chromosom, current_cost, idx_1, idx_2):
    new_chromosom = np.copy(current_chromosom)
    new_chromosom[idx_1], new_chromosom[idx_2] = new_chromosom[idx_2], new_chromosom[idx_1]

    mutationCost = current_cost - sum(np.sum(flow_matrix[[idx_1,idx_2], :] * dist_matrix[current_chromosom[[idx_1,idx_2], None], current_chromosom], 1))
    mutationCost += sum(np.sum(flow_matrix[[idx_1,idx_2], :] * dist_matrix[new_chromosom[[idx_1,idx_2], None], new_chromosom], 1))

    idx = list(range(chromosome_len)) ; del(idx[idx_1]) ; del(idx[idx_2-1])

    mutationCost -= sum(sum(flow_matrix[idx][:,[idx_1,idx_2]] * dist_matrix[current_chromosom[idx, None], current_chromosom[[idx_1,idx_2]]]))
    mutationCost += sum(sum(flow_matrix[idx][:,[idx_1,idx_2]] * dist_matrix[new_chromosom[idx, None], new_chromosom[[idx_1,idx_2]]]))

    return mutationCost

def cost(chromosome):
    return sum(np.sum(flow_matrix * dist_matrix[chromosome[:, None], chromosome], 1))

if __name__== "__main__": 
  ga_run_main()

[([0, 4, 5, 1, 2, 3], 102) ([4, 1, 0, 5, 2, 3], 108)
 ([0, 4, 5, 1, 3, 2], 110) ([0, 2, 3, 1, 4, 5], 112)
 ([1, 2, 3, 4, 0, 5], 112) ([2, 0, 3, 1, 4, 5], 112)
 ([0, 1, 4, 5, 3, 2], 118) ([2, 3, 1, 0, 5, 4], 118)
 ([1, 4, 5, 0, 2, 3], 120) ([1, 0, 2, 3, 5, 4], 122)
 ([3, 0, 4, 2, 5, 1], 122) ([5, 4, 3, 0, 1, 2], 122)
 ([3, 1, 0, 2, 4, 5], 124) ([4, 3, 5, 0, 1, 2], 124)
 ([0, 3, 4, 5, 1, 2], 126) ([1, 4, 0, 3, 2, 5], 126)
 ([2, 4, 0, 3, 5, 1], 126) ([2, 4, 3, 5, 1, 0], 126)
 ([2, 4, 5, 3, 0, 1], 126) ([4, 5, 0, 2, 3, 1], 128)
 ([2, 3, 4, 0, 5, 1], 130) ([3, 0, 4, 5, 1, 2], 130)
 ([3, 0, 2, 4, 5, 1], 134) ([5, 4, 2, 3, 0, 1], 134)
 ([2, 1, 4, 3, 0, 5], 136) ([2, 3, 4, 1, 5, 0], 136)
 ([3, 0, 5, 2, 1, 4], 136) ([0, 5, 3, 2, 1, 4], 138)
 ([0, 5, 3, 2, 1, 4], 138) ([3, 1, 4, 2, 5, 0], 138)
 ([4, 5, 2, 3, 0, 1], 138) ([4, 5, 2, 3, 0, 1], 138)
 ([5, 0, 3, 2, 1, 4], 138) ([0, 2, 1, 4, 5, 3], 140)
 ([4, 2, 3, 1, 5, 0], 140) ([5, 2, 1, 4, 0, 3], 140)
 ([0, 1, 3, 5, 4, 2], 142) ([1, 2, 0, 4, 5, 3]